In [4]:
import numpy as np
import pandas as pd
import sqlite3 as sql
import matplotlib.pyplot as plt
import warnings
from astropy import units as u
import pickle as pk
from pathlib import Path
import sys

from astroquery.jplhorizons import Horizons


from atm import modifyErrors
from atm import multiFit
from atm.models import NEATM
from atm.obs import WISE, WISE_ACT
from atm.analysis import calcMagChi2
from atm.analysis import calcMagReducedChi2
from atm.functions import calcFluxLambdaSED
from atm.plotting import plotObservations
from atm.plotting import plotSED

from asteroid_utils_atm import asteroid, get_desig

%load_ext autoreload
%autoreload 2

%matplotlib inline


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:

id_num = 0

desig, name, semimajor = get_desig(id_num)
flux_dir = '/scratch/r/rbond/jorlo/actxminorplanets/sigurd/fluxes/'

with open(flux_dir+ '{}_flux_dict.pk'.format(name), 'rb') as f:
    vesta_dict = pk.load(f)

vesta = asteroid(name, desig, vesta_dict, obs=WISE_ACT())
vesta.add_run('run0')

#Fit for 3 emmisitivites, one for each range
#Coppied from Run 4a but e3, e4 not fixed
run1 = {
    "fitParameters" : ["logT1", "logD", "eps_W1W2", 'eps_W3W4', 'eps_ACT'],
    "emissivitySpecification" : {
                "eps_W1W2" : ["W1","W2"],
                "eps_W3W4" : ["W3", "W4"],
                "eps_ACT" : ["090PA5", "090PA6", "150PA4", "150PA5", "150PA6", "220PA4"]},
    "albedoSpecification": "auto",
    "fitFilters" : "all",
    "columnMapping" : {
                "obs_id" : "obs_id",
                "designation" : "designation",
                "exp_mjd" : "mjd",
                "r_au" : "r_au",
                "delta_au" : "delta_au",
                "alpha_rad" : "alpha_rad",
                "eps" : ["eps_W3", "eps_W4"],
                "p" : None,
                "G" : "G",
                "logT1" : None,
                "logD" : None,
                "flux_si" : ["flux_W1_si", "flux_W2_si", "flux_W3_si", "flux_W4_si"],
                "fluxErr_si" : ["fluxErr_W1_si", "fluxErr_W2_si", "fluxErr_W3_si", "fluxErr_W4_si"],  
                "mag" : ["mag_W1", "mag_W2", "mag_W3", "mag_W4"],
                "magErr" : ["magErr_W1", "magErr_W2", "magErr_W3", "magErr_W4"]
    }
}

vesta.add_run('run1', runConfig = run1)


#Fit for 2 Temperatures, one for WISE one for ACT
#Coppied from Run 4a but e3, e4 not fixed
run2 = {
    "fitParameters" : ["logT1", 'logT1ACT', "logD", "eps_W1W2", 'eps_W3W4', 'eps_ACT'],
    "emissivitySpecification" : {
                "eps_W1W2" : ["W1","W2"],
                "eps_W3W4" : ["W3", "W4"],
                "eps_ACT" : ["090PA5", "090PA6", "150PA4", "150PA5", "150PA6", "220PA4"]},
    "albedoSpecification": "auto",
    "fitFilters" : "all",
    "columnMapping" : {
                "obs_id" : "obs_id",
                "designation" : "designation",
                "exp_mjd" : "mjd",
                "r_au" : "r_au",
                "delta_au" : "delta_au",
                "alpha_rad" : "alpha_rad",
                "eps" : None,#["eps_W3", "eps_W4"],
                "p" : None,
                "G" : "G",
                "logT1" : None,
                "logD" : None,
                "flux_si" : ["flux_W1_si", "flux_W2_si", "flux_W3_si", "flux_W4_si"],
                "fluxErr_si" : ["fluxErr_W1_si", "fluxErr_W2_si", "fluxErr_W3_si", "fluxErr_W4_si"],  
                "mag" : ["mag_W1", "mag_W2", "mag_W3", "mag_W4"],
                "magErr" : ["magErr_W1", "magErr_W2", "magErr_W3", "magErr_W4"]
    }
}

vesta.add_run('run2', runConfig = run2)
'''
run3 = {
    "fitParameters" : ["logT1", 'logT1ACT', "logD", "eps_WISE",  'eps_ACT'],
    "emissivitySpecification" : {
                "eps_WISE" : ["W1","W2", "W3", "W4"],
                "eps_ACT" : ["090PA5", "090PA6", "150PA4", "150PA5", "150PA6", "220PA4"]},
    "albedoSpecification": "auto",
    "fitFilters" : "all",
    "columnMapping" : {
                "obs_id" : "obs_id",
                "designation" : "designation",
                "exp_mjd" : "mjd",
                "r_au" : "r_au",
                "delta_au" : "delta_au",
                "alpha_rad" : "alpha_rad",
                "eps" : ["eps_W3", "eps_W4"],
                "p" : None,
                "G" : "G",
                "logT1" : None,
                "logD" : None,
                "flux_si" : ["flux_W1_si", "flux_W2_si", "flux_W3_si", "flux_W4_si"],
                "fluxErr_si" : ["fluxErr_W1_si", "fluxErr_W2_si", "fluxErr_W3_si", "fluxErr_W4_si"],  
                "mag" : ["mag_W1", "mag_W2", "mag_W3", "mag_W4"],
                "magErr" : ["magErr_W1", "magErr_W2", "magErr_W3", "magErr_W4"]
    }
}

vesta.add_run('run3', runConfig = run3)
'''

#vesta.atm_fit()
#vesta.make_SED()
#vesta.plt_SEDs()

'\nrun3 = {\n    "fitParameters" : ["logT1", \'logT1ACT\', "logD", "eps_WISE",  \'eps_ACT\'],\n    "emissivitySpecification" : {\n                "eps_WISE" : ["W1","W2", "W3", "W4"],\n                "eps_ACT" : ["090PA5", "090PA6", "150PA4", "150PA5", "150PA6", "220PA4"]},\n    "albedoSpecification": "auto",\n    "fitFilters" : "all",\n    "columnMapping" : {\n                "obs_id" : "obs_id",\n                "designation" : "designation",\n                "exp_mjd" : "mjd",\n                "r_au" : "r_au",\n                "delta_au" : "delta_au",\n                "alpha_rad" : "alpha_rad",\n                "eps" : ["eps_W3", "eps_W4"],\n                "p" : None,\n                "G" : "G",\n                "logT1" : None,\n                "logD" : None,\n                "flux_si" : ["flux_W1_si", "flux_W2_si", "flux_W3_si", "flux_W4_si"],\n                "fluxErr_si" : ["fluxErr_W1_si", "fluxErr_W2_si", "fluxErr_W3_si", "fluxErr_W4_si"],  \n                "mag" : ["mag_W1"

In [10]:
temp_summary = vesta.summary[vesta.summary['code']=='run1']

In [59]:
temp_summary[temp_summary['parameter'] == 'logT1']['median'].values[0] = temp_summary[temp_summary['parameter'] == 'logT1ACT']['median'].values[0]

In [11]:
temp_summary

In [9]:
vesta.data

,obs_id,designation,r_au,delta_au,alpha_deg,H_mag,G,mjd,flux_W1_si,fluxErr_W1_si,...,fluxErr_150PA5_si,fluxErr_150PA6_si,fluxErr_090PA5_si,fluxErr_090PA6_si,flux_090PA5_si,flux_090PA6_si,flux_150PA4_si,flux_150PA5_si,flux_150PA6_si,flux_220PA4_si
54192,54193,4,2.314689,1.987373,25.7343,3.2,0.32,55327.044366,5.156616e-07,9.403850e-08,...,9.018704e-15,8.636937e-15,2.496274e-15,2.490781e-15,7.787153e-14,7.936276e-14,4.783673e-13,4.739481e-13,4.635493e-13,1.945285e-12
54193,54194,4,2.314428,1.990280,25.7480,3.2,0.32,55327.308975,4.839103e-07,9.003096e-08,...,9.018704e-15,8.636937e-15,2.496274e-15,2.490781e-15,7.787153e-14,7.936276e-14,4.783673e-13,4.739481e-13,4.635493e-13,1.945285e-12
54194,54195,4,2.314363,1.991006,25.7514,3.2,0.32,55327.375063,5.384741e-07,1.155570e-07,...,9.018704e-15,8.636937e-15,2.496274e-15,2.490781e-15,7.787153e-14,7.936276e-14,4.783673e-13,4.739481e-13,4.635493e-13,1.945285e-12
54195,54196,4,2.313972,1.995369,25.7712,3.2,0.32,55327.771975,5.194752e-07,8.899251e-08,...,9.018704e-15,8.636937e-15,2.496274e-15,2.490781e-15,7.787153e-14,7.936276e-14,4.783673e-13,4.739481e-13,4.635493e-13,1.945285e-12
54196,54197,4,2.313711,1.998278,25.7840,3.2,0.32,55328.036584,5.633325e-07,1.011754e-07,...,9.018704e-15,8.636937e-15,2.496274e-15,2.490781e-15,7.787153e-14,7.936276e-14,4.783673e-13,4.739481e-13,4.635493e-13,1.945285e-12


In [10]:
fig, ax = plt.subplots(1, 1, dpi=vesta.DPI)
plotObservations(vesta.obs, vesta.data, columnMapping = vesta.columnMapping, ax=ax, plotMedian=True)
plt.show()

In [8]:
vesta.obs.filterEffectiveLambdas

In [43]:
obs.filterEffectiveLambdas[-6:]

In [24]:
vesta.summary

In [21]:
vesta.runDict['run0']

In [17]:
fig, ax = plt.subplots(1, 1, dpi=600)
plotObservations(vesta.obs, vesta.data, columnMapping=vesta.columnMapping, ax=ax, plotMedian=True)

In [ ]:
pallas_dict = {'090':{'flux':[139.01588, 131.57227], 'var':[2.0528338, 2.4021997]},
              '150':{'flux':[326.52725, 310.75455, 285.30325], 'var':[5.2094827, 2.691771, 3.0986135]},
              '220':{'flux':[573.7276], 'var':[6.848735]}
             }

pallas = asteroid('2', pallas_dict)
pallas.atm_fit()
pallas.make_SED()
pallas.plt_SEDs()

In [ ]:
interamnia_dict ={'090':{'flux':[26.955114, 26.414145], 'var':[2.979407, 3.5210254]},
              '150':{'flux':[71.74788, 65.685844, 56.5118], 'var':[5.9909983, 3.8527966, 4.1188393]},
              '220':{'flux':[118.91509], 'var':[8.563795]}
             }

interamnia = asteroid('704', interamnia_dict)
interamnia.atm_fit()
interamnia.make_SED()
interamnia.plt_SEDs()

In [ ]:
hygiea_dict ={'090':{'flux':[36.922863, 41.581024], 'var':[2.7536077, 3.6568952]},
              '150':{'flux':[93.648224, 100.43267, 84.54032], 'var':[7.0395627, 3.6568952, 4.767336]},
              '220':{'flux':[205.06685], 'var':[8.868363]}
             }

hygiea = asteroid('10', hygiea_dict)
hygiea.atm_fit()
hygiea.make_SED()
hygiea.plt_SEDs()

In [ ]:
def _get_desig(id_num):
    home = str(Path.home())
    with open(home+'/dev/minorplanets/asteroids.txt') as f:
        lines = f.readlines()
        name = lines[id_num].replace('\n', '')
        ast = Horizons(id=name)

        ast_name = ast.elements()['targetname'][0]
        desig = ''
        for i in range(len(ast_name)):
            if ast_name[i] == ' ': break
            desig += ast_name[i]
            
    return str(desig), name

def get_desig(id_num):
    home = str(Path.home())
    with open(home+'/dev/minorplanets/asteroids.pk', 'rb') as f:
        df = pk.load(f)
        name = df['name'][id_num]
        desig = df['designation'][id_num]
        semimajor = df['semimajor'][id_num]
    return desig, name, semimajor

class asteroid():
    def __init__(self, asteroid_name, desig, act_flux_dict, atm_dir = '/home/r/rbond/jorlo/dev/atm/',
            save_path = '/scratch/r/rbond/jorlo/actxminorplanets/sigurd/atm_fits/', 
            plot_path = '/scratch/r/rbond/jorlo/actxminorplanets/sigurd/plots/'):
        self.name = asteroid_name
        self.desig = desig
        self.act_flux_dict = act_flux_dict
        self.path = atm_dir
        self.save_path = save_path
        self.plot_path = plot_path
        
        self.columnMapping = {
                        "obs_id" : "obs_id",
                        "designation" : "designation",
                        "exp_mjd" : "mjd",
                        "r_au" : "r_au",
                        "delta_au" : "delta_au",
                        "alpha_rad" : "alpha_rad",
                        "eps" : None,
                        "p" : None,
                        "G" : "G",
                        "logT1" : None,
                        "logD" : None,
                        "flux_si": ["flux_W1_si", "flux_W2_si", "flux_W3_si", "flux_W4_si", "flux_090PA5_si", "flux_090PA6_si", "flux_150PA4_si", "flux_150PA5_si", "flux_150PA6_si",  "flux_220PA4_si"],
                        "fluxErr_si": ["fluxErr_W1_si", "fluxErr_W2_si", "fluxErr_W3_si", "fluxErr_W4_si", "fluxErr_090PA5_si", "fluxErr_090PA6_si", "fluxErr_150PA4_si", "fluxErr_150PA5_si", "fluxErr_150PA6_si",  "fluxErr_220PA4_si"],
                        "mag" : ["mag_W1", "mag_W2", "mag_W3", "mag_W4", "mag_090PA5_si", "mag_090PA6_si", "mag_150PA4_si", "mag_150PA5_si", "mag_150PA6_si",  "mag_220PA4_si"],
                        "magErr" : ["magErr_W1", "magErr_W2", "magErr_W3", "magErr_W4", "magErr_090PA5_si", "magErr_090PA6_si", "magErr_150PA4_si", "magErr_150PA5_si", "magErr_150PA6_si",  "magErr_220PA4_si"]
            }
        
        #Put your atm_data dir here
        con = sql.connect(atm_dir +"atm_data/paper1/sample.db")
        self.observations = pd.read_sql("""SELECT * FROM observations""", con)
        self.additional = pd.read_sql("""SELECT * FROM additional""", con)

        # Only keep clipped observations
        self.observations = self.observations[self.observations["keep"] == 1]
        self.additional = self.additional[self.additional["obs_id"].isin(self.observations["obs_id"].values)]

        # Remove missing H value, G value objects... 
        self.observations = self.observations[~self.observations["designation"].isin(['2010 AJ104', '2010 BM69', '2010 DZ64', '2010 EL27', '2010 EW144',
           '2010 FE82', '2010 FJ48', '2010 HK10', '2010 LE80'])]

        # Convert phase angle to radians
        self.observations["alpha_rad"] = np.radians(self.observations["alpha_deg"])
        self.ran_override = False

        # Initialize observatory 
        self.obs = WISE()

        if self.ran_override == False:
            self.observations = modifyErrors(self.observations, self.obs, sigma=0.15)
            self.ran_override = True
        else:
            print("No need to run this again!")

        self.DPI = 600
        self.SAVE_DIR = "../plots/"
        self.FIG_FORMAT = "png"

        self.SAVE_FIGS = False
        
    def inv_var(self, flux_dict, freq):

        
        num = 0
        denom = 0
        
        for i in range(len(flux_dict[freq]['flux'])):
            num += flux_dict[freq]['flux'][i]/flux_dict[freq]['var'][i]**2
            denom += 1/flux_dict[freq]['var'][i]**2
            
        return num/denom, denom

    def atm_fit(self):        
        runDict = {}
        dataDict = {}

        fitConfig = {
                "chains" : 20,
                "samples" : 3000,
                "burnInSamples": 500,
                "threads": 20,
                "scaling": 0.01,
                "plotTrace" : True,
                "plotCorner" : True,
                "progressBar" : True,
                "figKwargs" : {"dpi" : self.DPI}
            }

        self.data = self.observations[self.observations["designation"].isin([self.desig])]
        if self.data.empty:
            sys.exit('No Wise data for {}'.format(self.name))
            
        self.data_additional = self.additional[self.additional["obs_id"].isin(self.data["obs_id"].values)] # Contains published magnitudes

        dataDict["run0"] = self.data.copy()

        runDict["run0"] = {
            "fitParameters" : ["logT1", "logD", "eps"],
            "emissivitySpecification" : None,
            "albedoSpecification": "auto",
            "fitFilters" : "all",
            "columnMapping" : self.columnMapping
        }
        dataDict["run1"] = self.data.copy()
        dataDict["run1"]["eps_W3W4"] = np.ones(len(self.data)) * 0.9

        runDict["run1"] = {
            "fitParameters" : ["logT1", "logD", "eps_W1W2"],
            "emissivitySpecification" : {
                        "eps_W1W2" : ["W1","W2"],
                        "eps_W3W4" : ["W3","W4"]},
            "albedoSpecification": "auto",
            "fitFilters" : "all",
            "columnMapping" : self.columnMapping
        }

        dataDict["run2a"] = self.data.copy()
        dataDict["run2a"]["eps_W3"] = np.ones(len(self.data)) * 0.70
        dataDict["run2a"]["eps_W4"] = np.ones(len(self.data)) * 0.86

        runDict["run2a"] = {
            "fitParameters" : ["logT1", "logD", "eps_W1W2"],
            "emissivitySpecification" : {
                        "eps_W1W2" : ["W1","W2"],
                        "eps_W3" : ["W3"],
                        "eps_W4" : ["W4"]},
            "albedoSpecification": "auto",
            "fitFilters" : "all",
            "columnMapping" : self.columnMapping
        }
        dataDict["run2b"] = self.data.copy()
        dataDict["run2b"]["eps_W3"] = np.ones(len(self.data)) * 0.70
        dataDict["run2b"]["eps_W4"] = np.ones(len(self.data)) * 0.86

        runDict["run2b"] = {
            "fitParameters" : ["logT1", "logD", "eps_W1", "eps_W2"],
            "emissivitySpecification" : "perBand",
            "albedoSpecification": "auto",
            "fitFilters" : "all",
            "columnMapping" : {
                        "obs_id" : "obs_id",
                        "designation" : "designation",
                        "exp_mjd" : "mjd",
                        "r_au" : "r_au",
                        "delta_au" : "delta_au",
                        "alpha_rad" : "alpha_rad",
                        "eps" : ["eps_W3", "eps_W4"],
                        "p" : None,
                        "G" : "G",
                        "logT1" : None,
                        "logD" : None,
                        "flux_si" : ["flux_W1_si", "flux_W2_si", "flux_W3_si", "flux_W4_si"],
                        "fluxErr_si" : ["fluxErr_W1_si", "fluxErr_W2_si", "fluxErr_W3_si", "fluxErr_W4_si"],  
                        "mag" : ["mag_W1", "mag_W2", "mag_W3", "mag_W4"],
                        "magErr" : ["magErr_W1", "magErr_W2", "magErr_W3", "magErr_W4"]
            }
        }

        dataDict["run3a"] = self.data.copy()
        dataDict["run3a"]["eps_W3"] = np.ones(len(self.data)) * 0.76
        dataDict["run3a"]["eps_W4"] = np.ones(len(self.data)) * 0.93

        runDict["run3a"] = {
            "fitParameters" : ["logT1", "logD", "eps_W1W2"],
            "emissivitySpecification" : {
                        "eps_W1W2" : ["W1","W2"],
                        "eps_W3" : ["W3"],
                        "eps_W4" : ["W4"]},
            "albedoSpecification": "auto",
            "fitFilters" : "all",
            "columnMapping" : self.columnMapping
        }

        dataDict["run3b"] = self.data.copy()
        dataDict["run3b"]["eps_W3"] = np.ones(len(self.data)) * 0.76
        dataDict["run3b"]["eps_W4"] = np.ones(len(self.data)) * 0.93

        runDict["run3b"] = {
            "fitParameters" : ["logT1", "logD", "eps_W1", "eps_W2"],
            "emissivitySpecification" : "perBand",
            "albedoSpecification": "auto",
            "fitFilters" : "all",
            "columnMapping" : {
                        "obs_id" : "obs_id",
                        "designation" : "designation",
                        "exp_mjd" : "mjd",
                        "r_au" : "r_au",
                        "delta_au" : "delta_au",
                        "alpha_rad" : "alpha_rad",
                        "eps" : ["eps_W3", "eps_W4"],
                        "p" : None,
                        "G" : "G",
                        "logT1" : None,
                        "logD" : None,
                        "flux_si" : ["flux_W1_si", "flux_W2_si", "flux_W3_si", "flux_W4_si"],
                        "fluxErr_si" : ["fluxErr_W1_si", "fluxErr_W2_si", "fluxErr_W3_si", "fluxErr_W4_si"],  
                        "mag" : ["mag_W1", "mag_W2", "mag_W3", "mag_W4"],
                        "magErr" : ["magErr_W1", "magErr_W2", "magErr_W3", "magErr_W4"]
            }
        }
        
        dataDict["run4a"] = self.data.copy()
        dataDict["run4a"]["eps_W3"] = np.ones(len(self.data)) * 0.80
        dataDict["run4a"]["eps_W4"] = np.ones(len(self.data)) * 0.98

        runDict["run4a"] = {
            "fitParameters" : ["logT1", "logD", "eps_W1W2"],
            "emissivitySpecification" : {
                        "eps_W1W2" : ["W1","W2"],
                        "eps_W3" : ["W3"],
                        "eps_W4" : ["W4"]},
            "albedoSpecification": "auto",
            "fitFilters" : "all",
            "columnMapping" : self.columnMapping
        }
        
        dataDict["run4b"] = self.data.copy()
        dataDict["run4b"]["eps_W3"] = np.ones(len(self.data)) * 0.80
        dataDict["run4b"]["eps_W4"] = np.ones(len(self.data)) * 0.98

        runDict["run4b"] = {
            "fitParameters" : ["logT1", "logD", "eps_W1", "eps_W2"],
            "emissivitySpecification" : "perBand",
            "albedoSpecification": "auto",
            "fitFilters" : "all",
            "columnMapping" : self.columnMapping
        }

        dataDict["run5a"] = self.data.copy()
        dataDict["run5a"]["eps"] = np.ones(len(self.data)) * 0.9
        dataDict["run5a"]["p_W3"] = np.ones(len(self.data)) * 0.0
        dataDict["run5a"]["p_W4"] = np.ones(len(self.data)) * 0.0

        runDict["run5a"] = {
            "fitParameters" : ["logT1", "logD", "p_W1W2"],
            "emissivitySpecification" : None,
            "albedoSpecification": {
                        "p_W1W2" : ["W1", "W2"],
                        "p_W3" : ["W3"],
                        "p_W4" : ["W4"]},
            "fitFilters" : "all",
            "columnMapping" : self.columnMapping
        }

        dataDict["run5b"] = self.data.copy()
        dataDict["run5b"]["eps"] = np.ones(len(self.data)) * 0.9
        dataDict["run5b"]["p_W3"] = np.ones(len(self.data)) * 0.0
        dataDict["run5b"]["p_W4"] = np.ones(len(self.data)) * 0.0

        runDict["run5b"] = {
            "fitParameters" : ["logT1", "logD", "p_W1W2"],
            "emissivitySpecification" : None,
            "albedoSpecification": {
                        "p_W1W2" : ["W1", "W2"],
                        "p_W3" : ["W3"],
                        "p_W4" : ["W4"]},
            "fitFilters" : ["W1", "W3", "W4"],
            "columnMapping" : self.columnMapping
        }

        obs = self.obs
        self.model = NEATM(verbose=False, tableDir=self.path+'atm/models/tables/neatm/')

        self.summary, self.model_observations = multiFit(self.model, self.obs, dataDict, runDict, fitConfig, 
                                               saveDir=self.save_path + self.name)
        self.dataDict = dataDict
        self.runDict = runDict



    def post_process(self):
        observations_pp, model_observations_pp, observed_stats, model_stats = postProcess(obs, 
                                                                                      data,
                                                                                      model_observations, 
                                                                                      summary)

    def make_SED(self):
        lambdaRange=[1.0e-6, 30e-5]
        lambdaNum=250
        lambdaEdges=[3.9e-6, 6.5e-6, 18.5e-6]

        SEDs = {}
        for code in self.runDict.keys():
            SEDs[code] = calcFluxLambdaSED(self.model, self.obs, self.dataDict[code], 
                                           summary=self.summary[self.summary["code"] == code], 
                                           fitParameters=self.runDict[code]["fitParameters"],
                                           emissivitySpecification=self.runDict[code]["emissivitySpecification"],
                                           albedoSpecification=self.runDict[code]["albedoSpecification"],
                                           columnMapping=self.runDict[code]["columnMapping"],
                                           lambdaRange=lambdaRange,
                                           lambdaNum=lambdaNum,
                                           lambdaEdges=lambdaEdges,
                                           linearInterpolation=False)

        lambdaRange=[1.0e-4, 30e-2]
        lambdaNum=250
        lambdaEdges=[3.9e-6, 6.5e-6, 18.5e-6]

        SEDs_ACT = {}
        for code in self.runDict.keys():
            SEDs_ACT[code] = calcFluxLambdaSED(self.model, self.obs, self.dataDict[code], 
                                           summary=self.summary[self.summary["code"] == code], 
                                           fitParameters=self.runDict[code]["fitParameters"],
                                           emissivitySpecification=self.runDict[code]["emissivitySpecification"],
                                           albedoSpecification=self.runDict[code]["albedoSpecification"],
                                           columnMapping=self.runDict[code]["columnMapping"],
                                           lambdaRange=lambdaRange,
                                           lambdaNum=lambdaNum,
                                           lambdaEdges=lambdaEdges,
                                           linearInterpolation=False)
        self.WISE_SEDs = SEDs
        self.ACT_SEDs = SEDs_ACT
        
        for code in SEDs.keys():
            SEDs[code] = SEDs[code].append(SEDs_ACT[code])
            
        self.FULL_SEDs = SEDs
    
    def plt_SEDs(self):
        fig, ax = plt.subplots(1, 1, dpi=self.DPI)
        plotObservations(self.obs, self.data, ax=ax, plotMedian=True)
        for code in self.dataDict.keys():
            plotSED(self.FULL_SEDs[code], ax=ax, plotKwargs={"label": code})
        print(self.act_flux_dict)
        flux_pa4_150, err_pa4_150 = self.act_flux_dict['pa4']['150']['flux'], self.act_flux_dict['pa4']['150']['var']
        flux_pa4_220, err_pa4_220 = self.act_flux_dict['pa4']['220']['flux'], self.act_flux_dict['pa4']['220']['var']
        
        flux_pa5_090, err_pa5_090 = self.act_flux_dict['pa5']['090']['flux'], self.act_flux_dict['pa5']['090']['var']
        flux_pa5_150, err_pa5_150 = self.act_flux_dict['pa5']['150']['flux'], self.act_flux_dict['pa5']['150']['var']
        
        flux_pa6_090, err_pa6_090 = self.act_flux_dict['pa6']['090']['flux'], self.act_flux_dict['pa6']['090']['var']
        flux_pa6_150, err_pa6_150 = self.act_flux_dict['pa6']['150']['flux'], self.act_flux_dict['pa6']['150']['var']
        
        act_fluxes = [flux_pa5_090, flux_pa6_090, flux_pa4_150, flux_pa5_150, flux_pa6_150, flux_pa4_220]*u.mJy
        act_errs = [err_pa5_090, err_pa6_090, err_pa4_150, err_pa5_150, err_pa6_150, err_pa4_220]*u.mJy
        act_freqs = np.array([0.00333103, 0.00333103, 0.00199862, 0.00199862, 0.00199862, 0.00136269,])*u.m

        act_flux_units = np.zeros(len(act_fluxes))
        act_errs_units = np.zeros(len(act_fluxes))
        for i in range(len(act_fluxes)):
            act_flux_units[i] = ((2.99792458e+14 * act_fluxes[i].to(u.W*u.m**-2*u.Hz**-1)).value / (act_freqs[i].to(u.um)**2).value)
            act_errs_units[i] = ((2.99792458e+14 * act_errs[i].to(u.W*u.m**-2*u.Hz**-1)).value / (act_freqs[i].to(u.um)**2).value)

        ax.errorbar(act_freqs.to(u.um).value, 
                                act_flux_units, 
                                act_errs_units, 
                                fmt='o',
                                c="k",
                                ms=2,
                                capsize=1,
                                elinewidth=1)
        ax.legend()
        ax.set_xlim(1, 1e5)
        plt.savefig('{}_plot.pdf'.format(self.name))
        plt.show()
        
        self.act_fluxes = act_fluxes
        self.act_flux_units = act_flux_units


In [10]:
# Grab observations and additional data
con = sql.connect("/home/r/rbond/jorlo/dev/atm/atm_data/paper1/sample.db")
#con = sql.connect('/home/r/rbond/jorlo/dev/atm/atm_data/paper1/runs/sigma10/atm_results_run0.db')
observations = pd.read_sql("""SELECT * FROM observations""", con)
additional = pd.read_sql("""SELECT * FROM additional""", con)

# Only keep clipped observations
observations = observations[observations["keep"] == 1]
additional = additional[additional["obs_id"].isin(observations["obs_id"].values)]

# Remove missing H value, G value objects... 
observations = observations[~observations["designation"].isin(['2010 AJ104', '2010 BM69', '2010 DZ64', '2010 EL27', '2010 EW144',
   '2010 FE82', '2010 FJ48', '2010 HK10', '2010 LE80'])]

# Convert phase angle to radians
observations["alpha_rad"] = np.radians(observations["alpha_deg"])
ran_override = False

# Initialize observatory 
obs = WISE_ACT()

observations['mag_090PA5_si'] = np.ones(len(observations['obs_id']))*np.nan
observations['mag_090PA6_si'] = np.ones(len(observations['obs_id']))*np.nan
observations['mag_150PA4_si'] = np.ones(len(observations['obs_id']))*np.nan
observations['mag_150PA5_si'] = np.ones(len(observations['obs_id']))*np.nan
observations['mag_150PA6_si'] = np.ones(len(observations['obs_id']))*np.nan
observations['mag_220PA4_si'] = np.ones(len(observations['obs_id']))*np.nan

observations['magErr_090PA5_si'] = np.ones(len(observations['obs_id']))*np.nan
observations['magErr_090PA6_si'] = np.ones(len(observations['obs_id']))*np.nan
observations['magErr_150PA4_si'] = np.ones(len(observations['obs_id']))*np.nan
observations['magErr_150PA5_si'] = np.ones(len(observations['obs_id']))*np.nan
observations['magErr_150PA6_si'] = np.ones(len(observations['obs_id']))*np.nan
observations['magErr_220PA4_si'] = np.ones(len(observations['obs_id']))*np.nan

columnMapping = {
        "designation" : "designation",
        "obs_id": "obs_id",
        "exp_mjd": "mjd",
        "r_au": "r_au",
        "delta_au": "delta_au",
        "alpha_rad": "alpha_rad",
        "G": "G",
        "logD": "logD",
        "logT1" : "logT1",
        "eta": "eta",
        "eps": "eps",
        "flux_si": ["flux_W1_si", "flux_W2_si", "flux_W3_si", "flux_W4_si", "flux_090PA5_si", "flux_090PA6_si", "flux_150PA4_si", "flux_150PA5_si", "flux_150PA6_si",  "flux_220PA4_si"],
        "fluxErr_si": ["fluxErr_W1_si", "fluxErr_W2_si", "fluxErr_W3_si", "fluxErr_W4_si", "fluxErr_090PA5_si", "fluxErr_090PA6_si", "fluxErr_150PA4_si", "fluxErr_150PA5_si", "fluxErr_150PA6_si",  "fluxErr_220PA4_si"],
        "mag" : ["mag_W1", "mag_W2", "mag_W3", "mag_W4", "mag_090PA5_si", "mag_090PA6_si", "mag_150PA4_si", "mag_150PA5_si", "mag_150PA6_si",  "mag_220PA4_si"],
        "magErr" : ["magErr_W1", "magErr_W2", "magErr_W3", "magErr_W4", "magErr_090PA5_si", "magErr_090PA6_si", "magErr_150PA4_si", "magErr_150PA5_si", "magErr_150PA6_si",  "magErr_220PA4_si"]
        }




In [11]:
if ran_override == False:
    observations = modifyErrors(observations, obs, sigma=0.15, columnMapping=columnMapping)
    ran_override = True
else:
    print("No need to run this again!")

In [12]:
id_num = 0

desig, name, semimajor = get_desig(id_num)
flux_dir = '/scratch/r/rbond/jorlo/actxminorplanets/sigurd/fluxes/'

with open(flux_dir+ '{}_flux_dict.pk'.format(name), 'rb') as f:
    flux_dict = pk.load(f)
    
data = observations[observations["designation"].isin([desig])]    
    
flux_pa4_150, err_pa4_150 = flux_dict['night']['pa4']['150']['flux'], flux_dict['night']['pa4']['150']['var']
flux_pa4_220, err_pa4_220 = flux_dict['night']['pa4']['220']['flux'], flux_dict['night']['pa4']['220']['var']

flux_pa5_090, err_pa5_090 = flux_dict['night']['pa5']['090']['flux'], flux_dict['night']['pa5']['090']['var']
flux_pa5_150, err_pa5_150 = flux_dict['night']['pa5']['150']['flux'], flux_dict['night']['pa5']['150']['var']

flux_pa6_090, err_pa6_090 = flux_dict['night']['pa6']['090']['flux'], flux_dict['night']['pa6']['090']['var']
flux_pa6_150, err_pa6_150 = flux_dict['night']['pa6']['150']['flux'], flux_dict['night']['pa6']['150']['var']

act_fluxes = [flux_pa5_090, flux_pa6_090, flux_pa4_150, flux_pa5_150, flux_pa6_150, flux_pa4_220]*u.mJy
act_errs = [err_pa5_090, err_pa6_090, err_pa4_150, err_pa5_150, err_pa6_150, err_pa4_220]*u.mJy
act_freqs = np.array([0.00333103, 0.00333103, 0.00199862, 0.00199862, 0.00199862, 0.00136269])*u.m

act_flux_units = np.zeros(len(act_fluxes))
act_errs_units = np.zeros(len(act_fluxes))
for i in range(len(act_fluxes)):
    #Convert 
    #Note factor of 1e6 converts from micron to m
    act_flux_units[i] = ((2.99792458e+14 * act_fluxes[i].to(u.W*u.m**-2*u.Hz**-1)).value / (act_freqs[i].to(u.um)**2).value)*1e6
    act_errs_units[i] = ((2.99792458e+14 * act_errs[i].to(u.W*u.m**-2*u.Hz**-1)).value / (act_freqs[i].to(u.um)**2).value)*1e6


In [13]:
flux_090PA5_si = np.ones(len(data['obs_id']))*act_flux_units[0]
flux_090PA6_si = np.ones(len(data['obs_id']))*act_flux_units[1]
flux_150PA4_si = np.ones(len(data['obs_id']))*act_flux_units[2]
flux_150PA5_si = np.ones(len(data['obs_id']))*act_flux_units[3]
flux_150PA6_si = np.ones(len(data['obs_id']))*act_flux_units[4]
flux_220PA4_si = np.ones(len(data['obs_id']))*act_flux_units[5]

fluxErr_090PA5_si = np.ones(len(data['obs_id']))*act_errs_units[0]
fluxErr_090PA6_si = np.ones(len(data['obs_id']))*act_errs_units[1]
fluxErr_150PA4_si = np.ones(len(data['obs_id']))*act_errs_units[2]
fluxErr_150PA5_si = np.ones(len(data['obs_id']))*act_errs_units[3]
fluxErr_150PA6_si = np.ones(len(data['obs_id']))*act_errs_units[4]
fluxErr_220PA4_si = np.ones(len(data['obs_id']))*act_errs_units[5]

data['flux_090PA5_si'] = flux_090PA5_si
data['flux_090PA6_si'] = flux_090PA6_si
data['flux_150PA4_si'] = flux_150PA4_si
data['flux_150PA5_si'] = flux_150PA5_si
data['flux_150PA6_si'] = flux_150PA6_si
data['flux_220PA4_si'] = flux_220PA4_si

data['fluxErr_090PA5_si'] = fluxErr_090PA5_si
data['fluxErr_090PA6_si'] = fluxErr_090PA6_si
data['fluxErr_150PA4_si'] = fluxErr_150PA4_si
data['fluxErr_150PA5_si'] = fluxErr_150PA5_si
data['fluxErr_150PA6_si'] = fluxErr_150PA6_si
data['fluxErr_220PA4_si'] = fluxErr_220PA4_si


In [ ]:
fig, ax = plt.subplots(1, 1, dpi=600)
plotObservations(obs, data, columnMapping=columnMapping, ax=ax, plotMedian=True)